In [ ]:

# استيراد المكتبات اللازمة
import streamlit as st
import pandas as pd
import re
import plotly.express as px
from datetime import datetime

# تهيئة متغيرات الحالة في الجلسة
if "expenses" not in st.session_state:
    st.session_state.expenses = []

if "budgets" not in st.session_state:
    st.session_state.budgets = {}

# تعريف كلمات مفتاحية مرتبطة بالتصنيفات
merchant_keywords = {
    "Amazon": "Shopping",
    "Hungerstation": "Food",
    "Uber": "Transport",
    "Alothaim": "Shopping",
    "Jahez": "Food",
    "Webook": "Entertainment",
    "Keeta": "Food",
    "Jarir": "Shopping",
    "Bolt": "Transport",
    "Aldrees": "Fuel",
}

# دالة لاستخراج بيانات المصروف من نص الرسالة
def extract_expense_from_sms(sms, date=None):
    # أولاً نحاول مطابقة التجار الموجودين في القائمة المعرفة
    for keyword in merchant_keywords:
        if keyword.lower() in sms.lower():
            amount_match = re.search(r'SAR[.\s]?\s?(\d+\.?\d*)', sms, re.IGNORECASE)
            if amount_match:
                amount = float(amount_match.group(1))
                return {
                    "description": f"Auto: {keyword}",
                    "category": merchant_keywords[keyword],
                    "amount": amount,
                    "date": date or datetime.today().strftime('%Y-%m-%d')
                }

    # إذا لم يتطابق التاجر مع القائمة، نحاول استخراج اسم التاجر باستخدام الأنماط
    amount_match = re.search(r'SAR[.\s]?\s?(\d+\.?\d*)', sms, re.IGNORECASE)
    if amount_match:
        amount = float(amount_match.group(1))

        merchant_patterns = [
            r'at\s+([A-Za-z\s]+)',
            r'to\s+([A-Za-z\s]+)',
            r'for\s+([A-Za-z\s]+)',
            r'order of\s+([A-Za-z\s]+)',
            r'purchase of\s+([A-Za-z\s]+)',
            r'charge of\s+([A-Za-z\s]+)',
            r'on\s+([A-Za-z\s]+)',
            r'from\s+([A-Za-z\s]+)',
        ]

        merchant = None
        for pattern in merchant_patterns:
            match = re.search(pattern, sms, re.IGNORECASE)
            if match:
                # إذا أردت استخراج أكثر من كلمة  (مثلاً "Al Uthaim") فحذف .split()[0]
                merchant = match.group(1).strip()
                break

        # لو لم نستخرج اسم تاجر من الأنماط، يمكننا محاولة استخراج الكلمة التالية للمبلغ
        if not merchant:
            idx = amount_match.end()
            tail = sms[idx:]
            match2 = re.search(r'\b([A-Za-z]{3,})\b', tail)
            if match2:
                merchant = match2.group(1)

        # إذا ما زلنا لم نستخرج اسم، نخليه فارغ أو "Unknown Merchant"
        if not merchant:
            merchant = "Unknown Merchant"

        return {
            "description": f"Auto: {merchant}",
            "category": "Uncategorized",
            "amount": amount,
            "date": date or datetime.today().strftime('%Y-%m-%d')
        }

    return None
# دالة لإضافة مصروف يدويا
def add_expense(description, category, amount, date):
    st.session_state.expenses.append({
        "description": description,
        "category": category,
        "amount": amount,
        "date": date
    })

# دالة لحساب إجمالي الإنفاق حسب التصنيف
def get_spending_summary():
    summary = {}
    for expense in st.session_state.expenses:
        cat = expense["category"]
        summary[cat] = summary.get(cat, 0) + expense["amount"]
    return summary

# دالة لإعطاء نصيحة بناءً على الإنفاق
def spending_advice(summary):
    high_spenders = [cat for cat, amt in summary.items() if amt > 100]
    if not high_spenders:
        return "Great job keeping your spending in control!"
    return "Watch out for high spending in: " + ", ".join(max(high_spenders))

# دالة لفحص تجاوز الميزانية المحددة
def check_budget_alerts(summary):
    alerts = []
    for category, spent in summary.items():
        budget = st.session_state.budgets.get(category)
        if budget and spent > budget:
            alerts.append(f"⚠️ You've exceeded your budget for {category}: Spent {spent} (Budget {budget})")
    return alerts

# عنوان التطبيق
st.title("Advanced Smart Expense Analyzer 💸")

# قسم إعداد الميزانية في الشريط الجانبي
st.sidebar.header("🔔 Set Monthly Budgets")
categories = ["Food", "Transport", "Entertainment", "Bills", "Shopping", "Other"]
for cat in categories:
    budget = st.sidebar.number_input(f"Set budget for {cat}", min_value=0.0, step=50.0, value=st.session_state.budgets.get(cat, 0.0))
    st.session_state.budgets[cat] = budget

# رفع ملف رسائل البنك بصيغة CSV
st.subheader("📂 Upload your bank SMS CSV (one message per row):")
uploaded_file = st.file_uploader("Choose a file", type="csv")

if uploaded_file:
    df = pd.read_csv(uploaded_file, header=None)
    sms_messages = df[0].tolist()
    for sms in sms_messages:
        extracted = extract_expense_from_sms(sms)
        if extracted:
            st.session_state.expenses.append(extracted)
    st.success("✅ Expenses extracted from SMS and added!")

# قسم إضافة مصروف يدويًا
st.subheader("📝 Add Expense Manually")
with st.form("manual_form"):
    description = st.text_input("Description")
    category = st.selectbox("Category", categories)
    amount = st.number_input("Amount ($)", min_value=0.0, step=0.5)
    date = st.date_input("Date", datetime.today())
    submitted = st.form_submit_button("Add Expense")
    if submitted:
        add_expense(description, category, amount, date.strftime('%Y-%m-%d'))
        st.success("✅ Manual expense added!")

# عرض المصروفات والتقارير إذا كانت هناك بيانات
if st.session_state.expenses:
    st.subheader("📋 All Expenses")
    df_exp = pd.DataFrame(st.session_state.expenses)
    df_exp["date"] = pd.to_datetime(df_exp["date"])
    st.dataframe(df_exp)

    summary = get_spending_summary()
    sorted_summary = dict(sorted(summary.items(), key=lambda x: x[1], reverse=True))

    # عرض مصروفات حسب التصنيف (مخطط دائري)
    st.subheader("📊 Spending by Category (Pie Chart)")
    pie_df = pd.DataFrame(list(sorted_summary.items()), columns=["Category", "Amount"])
    fig = px.pie(pie_df, names="Category", values="Amount", title="Spending by Category")
    st.plotly_chart(fig)

    # عرض مصروفات حسب الزمن (مخطط شريطي)
    st.subheader("📈 Spending Over Time")
    time_df = df_exp.copy()
    time_df["month"] = time_df["date"].dt.to_period("M").astype(str)
    month_summary = time_df.groupby(["month", "category"])["amount"].sum().reset_index()
    fig2 = px.bar(month_summary, x="month", y="amount", color="category", title="Monthly Spending by Category", barmode="group")
    st.plotly_chart(fig2)

    # عرض نصيحة مالية
    st.subheader("💡 Advice")
    st.info(spending_advice(summary))

    # عرض تنبيهات الميزانية
    st.subheader("🚨 Budget Alerts")
    alerts = check_budget_alerts(summary)
    if alerts:
        for alert in alerts:
            st.error(alert)
    else:
        st.success("✅ You are within your budgets for all categories!")
else:
    st.info("🛒 Add some expenses manually or upload SMS to get started.")